In [2]:
%cd /content
!git clone -b totoro2 https://github.com/camenduru/ComfyUI /content/TotoroUI
%cd /content/TotoroUI
!apt -y install -qq aria2

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/adamo1139/stable-diffusion-3-medium-ungated/resolve/main/sd3_medium_incl_clips_t5xxlfp8.safetensors -d /content/TotoroUI/model -o sd3_medium_incl_clips_t5xxlfp8.safetensors

import torch
import random
import node_helpers
from totoro.sd import load_checkpoint_guess_config
import nodes
import numpy as np
from PIL import Image

with torch.inference_mode():
    model_patcher, clip, vae, clipvision = load_checkpoint_guess_config("/content/TotoroUI/model/sd3_medium_incl_clips_t5xxlfp8.safetensors", output_vae=True, output_clip=True, embedding_directory=None)

[Errno 2] No such file or directory: '/content'
/kaggle/working
Cloning into '/content/TotoroUI'...
remote: Enumerating objects: 11621, done.
remote: Total 11621 (delta 0), reused 0 (delta 0), pack-reused 11621
Receiving objects: 100% (11621/11621), 5.72 MiB | 18.36 MiB/s, done.
Resolving deltas: 100% (7886/7886), done.
/content/TotoroUI
The following additional packages will be installed:
  libaria2-0 libc-ares2 libssh2-1
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2 libssh2-1
0 upgraded, 4 newly installed, 0 to remove and 75 not upgraded.
Need to get 1551 kB of archives.
After this operation, 6235 kB of additional disk space will be used.

78Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 113807 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.15.0-1ubuntu0.5_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [  6%] [###.............

In [3]:

import langchain
from langchain import PromptTemplate

import nest_asyncio
nest_asyncio.apply()


In [7]:
from PIL import ImageDraw, ImageFont
from string import ascii_letters
import textwrap
import requests
from telegram import ReplyKeyboardMarkup
import sqlite3
from datetime import datetime
from telegram import Update,InlineKeyboardButton, InlineKeyboardMarkup,  InlineQueryResultArticle, InputTextMessageContent
from telegram.ext import filters,  MessageHandler, ApplicationBuilder,CallbackQueryHandler, CommandHandler, ContextTypes, ConversationHandler,InlineQueryHandler
import nest_asyncio
# Define the states for the conversat
ADD_TEXT, STYLE, PREVIEW, GET_CAPTION, ASK_CAPTION, CANCELCONFIRM, ASK_DOWNLOAD, DOWNLOAD = range(8)






# Database connection
conn = sqlite3.connect('tele_bot_data.db', check_same_thread=False)
cursor = conn.cursor()


# Create feedback table if it doesn't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS feedback (
        user_id INTEGER PRIMARY KEY,
        feedback TEXT,
        timestamp TEXT
    )
''')
conn.commit()

# adding functionality
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    print('started start ')
    user = update.effective_user

    await context.bot.send_animation(
        chat_id = update.effective_chat.id,
        animation='CAACAgIAAxkBAAETiwABZoxOHE2Mfq5elov-LTRAM8_UGLMAAgUAA8A2TxP5al-agmtNdTUE'
    )
    await update.message.reply_text('Hello!  \nUse /help to see available commands.\n\nUse /addtext to generate your own custom stickers. \n\nOr automatically generate stickers based on these trending topics!')
    keyboard6 = [
        [InlineKeyboardButton("Cricket World Cup", callback_data='Cricket World Cup'),InlineKeyboardButton("Paris Olympics", callback_data='Paris Olympics')],
        [InlineKeyboardButton("Raksha Bandhan", callback_data='Raksha Bandhan'),InlineKeyboardButton("Friendship Day", callback_data='Friendship Day')],
    ]
    reply_markup6 = InlineKeyboardMarkup(keyboard6)
    await update.message.reply_text('*Trending stickers:*', reply_markup=reply_markup6, parse_mode='Markdown')
    print('start return trending 9 or add text')
    return 1

async def trending(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    print('started trending')
    query = update.callback_query
    await query.answer()
    context.user_data['trending'] = query.data
    trending=query.data
    print(f'user chose {trending} ')

    if trending =="Cricket World Cup":
        detail="A photorealistic image of Indian cricketers in iconic blue jerseys, jumping for joy and embracing each other in the center of the field. The players are surrounded by a subtle, blurred suggestion of a packed stadium, with faint hints of:Ecstatic fans in the stands, waving flags and cheering, Fireworks illuminating the night sky, Giant screens displaying the winning score and celebratory messages. The focus is on the players\' triumphant celebration, with the stadium atmosphere reduced to subtle, contextual elements that don\'t disrupt the main motive when background removal is applied."
        caption_text="Champions"
        font_color= (19,57,132)

    elif trending =="Paris Olympics":
        detail="Olympic rings (blue, yellow, black, green, red) MAKE SURE ALL 5 RINGS PRESENT, Paris 2024 logo (beige flame as Marianne), Paris landmarks (Eiffel Tower) Athletes in action (various Olympic sports, dynamic poses)MAKE SURE EVERTHING IS INSIDE A CIRCLE BORDER OF WHITE COLOR"
        caption_text="Games wide open"
        font_color= (199,142,29)

    elif trending =="Raksha Bandhan":
        detail="A circle(with white border) cartoonish image (big eyes and face) in a bright and cheerful style for kids! It shows a Rakshabandhan celebration in a tastefully decorated  home. A small girl with a big smile, gleteering eyes, wearing a indian traditional outfit , ties a colorful flowery big  rakhi on her  small brother's wrist . He's dressed in traditional kurta pajama. The house is decorated with festive details - swirly patterns on the floor (rangolis) and flower chains (marigold garlands) everywhere. A table overflows with yummy treats (ladoos, barfi, and samosas)! Warm light bathes the scene, highlighting the love and respect between the siblings. MAKE SURE EVERYTHING IS INSIDE THE WHITE BORDER"
        caption_text="Happy Raksha Bandhan"
        font_color= "red"

    elif trending =="Friendship Day":
        detail="A vibrant, cartoon-style illustration of a fun-loving group of friends, surrounded by confetti, balloons, and colorful streamers! They\'re all gathered around a giant, sparkling FRIENDSHIP banner, beaming with joy and excitement. Each friend is uniquely designed with exaggerated features, bright colors, and playful expressions. They\'re all engaged in fun activities that showcase their friendship, like: Sharing a giant ice cream sundae, Playing a game of high-five tag, Exchanging silly, colorful gifts, The background is a bright, sunny blue with fluffy white clouds, representing a day filled with laughter and adventure. The overall atmosphere is electric, with a sense of excitement and celebration! with a sprinkle of emojis and glitter effects to match the cheerful vibe."
        caption_text="Friends Forever"
        font_color= (247,136,19)

    await context.bot.send_message(chat_id=update.effective_chat.id, text=f' *Theme chosen:* _{trending}_' , parse_mode='Markdown')

    await query.edit_message_text(text='Your sticker is being created....\nIt will only take a minute')
    loading_message = await context.bot.send_animation(
        chat_id=query.message.chat_id,
        animation='CAACAgIAAxkBAAETiuhmjEZTBfGfx_F5LGU1DFs-RFGXfAACBgADwDZPE8fKovSybnB2NQQ'
    )


    def zero_out(conditioning):
        c = []
        for t in conditioning:
            d = t[1].copy()
            if "pooled_output" in d:
                d["pooled_output"] = torch.zeros_like(d["pooled_output"])
            n = [torch.zeros_like(t[0]), d]
            c.append(n)
        return (c, )

    with torch.inference_mode():
        latent = {"samples": torch.ones([1, 16, 1024 // 8, 1024 // 8]) * 0.0609}
        prompt = detail
        cond, pooled = clip.encode_from_tokens(clip.tokenize(prompt), return_pooled=True)
        cond = [[cond, {"pooled_output": pooled}]]
        negative_prompt = "extra body parts,bad quality, poor quality, disfigured, jpg, toy, bad anatomy, missing limbs, missing fingers, 3d, cgi"
        n_cond, n_pooled = clip.encode_from_tokens(clip.tokenize(negative_prompt), return_pooled=True)
        n_cond = [[n_cond, {"pooled_output": n_pooled}]]

        n_cond1 = node_helpers.conditioning_set_values(n_cond, {"start_percent": 0, "end_percent": 0.1})
        n_cond2 = zero_out(n_cond)
        n_cond2 = node_helpers.conditioning_set_values(n_cond2[0], {"start_percent": 0.1, "end_percent": 1.0})
        n_cond = n_cond1 + n_cond2

        seed = 1
        if seed == 0:
            seed = random.randint(0, 18446744073709551615)
        print(seed)
        sample = nodes.common_ksampler(model=model_patcher,
                                seed=seed,
                                steps=28,
                                cfg=4.5,
                                sampler_name="dpmpp_2m",
                                scheduler="sgm_uniform",
                                positive=cond,
                                negative=n_cond,
                                latent=latent,
                                denoise=1)
        sample = sample[0]["samples"].to(torch.float16)
        vae.first_stage_model.cuda()
        decoded = vae.decode_tiled(sample).detach()


    Image1 = Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0])
    Image1.save("cutie2.png")
    image_path = '/content/TotoroUI/cutie2.png'
    cutie = Image.open(image_path)
    cutie = cutie.resize((512, 512))
        

    def round_corner(radius, fill):
        corner = Image.new('RGB', (radius, radius), (0, 0, 0, 0))
        draw = ImageDraw.Draw(corner)
        draw.pieslice((0, 0, radius * 2, radius * 2), 180, 270, fill=fill)
        return corner

    def round_rectangle(size, radius, fill):
        """Draw a rounded rectangle"""
        width, height = size
        rectangle = Image.new('RGBA', size, fill)
        corner = round_corner(radius, fill)
        rectangle.paste(corner, (0, 0))
        rectangle.paste(corner.rotate(90), (0, height - radius))  # Rotate the corner and paste it
        rectangle.paste(corner.rotate(180), (width - radius, height - radius))
        rectangle.paste(corner.rotate(270), (width - radius, 0))
        return rectangle



    def get_font_size(textarea, text, font_name, pixel_gap=5, padding=10):
        text_width = textarea[0] - 2 * padding
        text_height = textarea[1]
        for point_size in range(90, 5, -1):
            font = ImageFont.truetype(font_name, point_size)
            avg_char_width = sum(font.getbbox(char)[2] for char in ascii_letters) / len(ascii_letters)
            max_char_height = max(font.getbbox(char)[3] - font.getbbox(char)[1] for char in ascii_letters)

            max_char_count = int(text_width / avg_char_width)
            wrapped_text = textwrap.fill(text=text, width=max_char_count)
            num_lines = len(wrapped_text.splitlines())

            if (max_char_height * num_lines) + (pixel_gap * (num_lines + 1)) <= text_height:
                return wrapped_text, point_size

        return "", 5  # Fallback to minimum size if no suitable size is found

    
    imagee = Image.open(image_path)
    color = (255, 255, 255, 255)
    caption_area = (1024, 100)
    padding = 0  # Adjust this value to increase/decrease padding
    caption5 = round_rectangle((1024, 100), 10, (255,255,255,255))
    text = caption_text
    font_name = "" #(path to directory/comicbd.ttf)  # Ensure the font file is in the same directory as your script

    # Get wrapped text and appropriate font size
    wrapped_text, font_size = get_font_size(caption_area, text, font_name, padding=padding)

    # Draw text onto the caption
    draw = ImageDraw.Draw(caption5)
    font = ImageFont.truetype(font_name, font_size)

    # Center the text vertically and horizontally with padding
    text_bbox = draw.textbbox((0, 0), wrapped_text, font=font)
    text_width = text_bbox[2] - text_bbox[0]
    text_height = text_bbox[3] - text_bbox[1]
    x = padding + (caption_area[0] - 2 * padding - text_width) // 2
    y = (caption_area[1] - text_height-20) // 2
    draw.text((x, y), wrapped_text, fill=font_color, font=font)

    # Paste the caption onto the image
    imagee.paste(caption5, (0, 924), caption5)
    imagee.save('cutie2.png')
    img_path = '/content/TotoroUI/cutie2.png'
    with open(img_path, 'rb') as sticker_file:
        await context.bot.send_sticker(chat_id=update.effective_chat.id, sticker=sticker_file)


    await context.bot.delete_message(
        chat_id=loading_message.chat_id,
        message_id=loading_message.message_id
    )

    await query.edit_message_text("Here\'s your sticker :)  Thank you for your patience." )
    await context.bot.send_message(chat_id=update.effective_user.id, text="Use /download to download the sticker as image file")
    print('trending return ask download 7')
    return 7


async def helps(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    help_text = (
        "/start - Start creating a sticker\n"
        "/addtext - to add description for your sticker\n"
        "once you add description you will have following options:\n"
        "     /style - Choose a style for your sticker\n"
        "     /preview - Preview your sticker\n"
        "     /confirm - Finalize and generate your new sticker\n"
        "     /cancel - Cancel the current sticker creation process\n"
        "You can also download sticker as image file in your device\n"
        "/feedback - Provide feedback on generated stickers\n"
        "/rate - Rate your experience with the bot\n"

    )
    await update.message.reply_text(help_text)

async def addtext(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    print("started add text")
    await update.message.reply_text('*Please enter the textual description for your sticker now:*', parse_mode='Markdown')
    print("add text return receive text")
    return 0#RECEIVE_TEXT

async def receive_text(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    print("receive text started")
    context.user_data['description'] = update.message.text
    description = update.message.text
    await update.message.reply_text(f'*Your description: *_{description}_\n\nNow, Use /style to choose a style for your sticker.', parse_mode='Markdown')
    print("receive text return style 1")
    return 1#STYLE


# Style handler

async def style(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    print("style started")
    keyboard1 = [
        [InlineKeyboardButton("Neon", callback_data='neon'),InlineKeyboardButton("Dreamy", callback_data='dreamy')],
        [InlineKeyboardButton("Realistic", callback_data='realistic'),InlineKeyboardButton("Doodle", callback_data='doodle')],
        [InlineKeyboardButton("Anime", callback_data='anime'),InlineKeyboardButton("Pixel art", callback_data='pixel art')],
        [InlineKeyboardButton("Watercolor", callback_data='watercolor'),InlineKeyboardButton("Pastel", callback_data='pastel')],
        [InlineKeyboardButton("Retro", callback_data='retro'),InlineKeyboardButton("Vibrant", callback_data='vibrant')],
        [InlineKeyboardButton("Mosaic", callback_data='mosaic'),InlineKeyboardButton("Oil Painting", callback_data='oil painting')]
    ]
    reply_markup1 = InlineKeyboardMarkup(keyboard1)
    await update.message.reply_text('*Please choose a style:*', reply_markup=reply_markup1, parse_mode='Markdown')
    print("style return receive style")
    return 2#STYLE

# Asynchronous callback query handler for style selection
async def receive_style(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    print("receive style started")
    query = update.callback_query
    await query.answer()
    context.user_data['style'] = query.data
    style = query.data
    keyboard2 = [
        [InlineKeyboardButton("Yes", callback_data='yes')],
        [InlineKeyboardButton("No", callback_data='no')],
    ]
    reply_markup2 = InlineKeyboardMarkup(keyboard2)
    await query.edit_message_text(text=f' *Style choosen: * _{style}_ ' , parse_mode='Markdown')
    await context.bot.send_message(chat_id=query.message.chat_id,text="*Do you want to add caption text to your sticker?*" , reply_markup=reply_markup2, parse_mode='Markdown')
    print("receive style returns ask caption 3")
    return 3# ASK_CAPTION

async def ask_caption(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    print("ask caption started")
    query = update.callback_query
    await query.answer()
    global want_cap
    want_cap = query.data
    if want_cap == 'yes':
        await query.edit_message_text(text="*Enter your caption:*", parse_mode='Markdown')
        print("ask caption returns get caption")
        return 4#GET_CAPTION
    else:
        await query.edit_message_text(text="_No caption chosen._ \nUse /preview to preview your sticker", parse_mode= 'Markdown')
        print("ask caption returns preview")
        return 5#PREVIEW

async def get_caption(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    print("get caption started")
    caption = update.message.text
    context.user_data['caption'] = caption
    await update.message.reply_text(f"*Caption received:* _{caption}_ \nUse /preview to preview your sticker", parse_mode= 'Markdown')
    print("get caption returns preview")
    return 5#PREVIEW




# Asynchronous command handler for /preview
async def preview(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    print("preview started")
    description = context.user_data.get('description', 'No description')
    style = context.user_data.get('style', 'No style')
    caption_text = context.user_data.get('caption', 'No caption')
    keyboard3 = [
        [InlineKeyboardButton("Confirm", callback_data='confirm')],
        [InlineKeyboardButton("Cancel", callback_data='cancel')],
    ]
    reply_markup3 = InlineKeyboardMarkup(keyboard3)
    await update.message.reply_text(f' Preview: \n *Description = * _{description}_ \n *Style =* _{style}_ \n *Caption text = * _{caption_text}_ ', parse_mode='Markdown')
    await update.message.reply_text(text="*Do you want to continue?*" , reply_markup=reply_markup3, parse_mode='Markdown')
    print("preview returns cancel confirm")
    return 6#CANCELCONFIRM


async def cancel_confirm(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    print("cancel confirm started")
    query = update.callback_query
    await query.answer()
    if query.data == 'confirm':
        print("user clicked confirm")
        await query.edit_message_text('*Sticker confirmed!* Your sticker is being created....\nIt will only take a minute', parse_mode='Markdown')
        loading_message = await context.bot.send_animation(
            chat_id=query.message.chat_id,
            animation='CAACAgIAAxkBAAETiuhmjEZTBfGfx_F5LGU1DFs-RFGXfAACBgADwDZPE8fKovSybnB2NQQ'
        )

        user_id = update.effective_user.id
        description = context.user_data.get('description', 'No description')
        style = context.user_data.get('style', 'No style')
        if style=="neon":
            detail_style="iridescent illustration, neon glowing lights"
        elif style=="realistic":
            detail_style="a highly detailed, photorealistic portrait , highly detailed, photorealistic, or accurate portrayal, realistic depth," "accurate perspective, realistic lighting ,highlights and shadows, natural blemishes, realistic skin tones,"
        elif style=="dreamy":
            detail_style="detailed, fantasy, cute, adorable, Pixar, Disney, 8k,"
        elif style=="anime":
            detail_style="anime, chibi, studio ghilbi, Large, emotive eyes, Think sparkling eyes for joy, furrowed brows for anger, or teary eyes for sadness,utilizes a bold and saturated color palette, with bright hair, clothing, dynamic poses,Certain features, like eyes, hair, or limbs, might be exaggerated for dramatic effect"
        elif style=="doodle":
            detail_style=" a detailed doodle art illustration which is hand-drawn- sketchy, clear shapes, bold outlines, playful art style, unrefined, or loose lines, whimsical details, squiggles, fun flourishes,"
        elif style=="pixel art":
            detail_style="a detailed 8 bit pixel art illustration, pixelated full body, character icon concept art, Pixel Perfect, Use a vibrant 16-color palette with chunky black outlines, classic video game look,"
        elif style=="pastel":
            detail_style="pastel holographic iridescent illumination, light colors,soft, delicate, pastel drawings  that incorporate pastel colors and subtle shading,Cottagecore, disney, pixar, cute, adorable, 8k,"
        elif style=="mosaic":
            detail_style="mosaic art style, composed of small Tiles,"
        elif style=="retro":
            detail_style="retro, vintage, graphic novel styles, beige undertones,"
        elif style=="vibrant":
            detail_style="vibrant, electric colours, high contrast, pop art style,"
        elif style=="watercolor":
            detail_style="watercolour based art, soft, fluid appearance, resembling traditional watercolor painting,"
        elif style=="oil painting":
            detail_style="in oil painting style, realistic lighting, visible brushstrokes, luminous colors, and a charming painterly aesthetic that is created with oil paint,"


        def zero_out(conditioning):
            c = []
            for t in conditioning:
                d = t[1].copy()
                if "pooled_output" in d:
                    d["pooled_output"] = torch.zeros_like(d["pooled_output"])
                n = [torch.zeros_like(t[0]), d]
                c.append(n)
            return (c, )

        with torch.inference_mode():
            latent = {"samples": torch.ones([1, 16, 1024 // 8, 1024 // 8]) * 0.0609}
            prompt = PromptTemplate(
            input_variables=["descr","sty"],
            template="digital sticker of {descr}, {sty},centered ,must be a solid dark grey background")

            input_text = prompt.format(descr=description,sty=detail_style)
            cond, pooled = clip.encode_from_tokens(clip.tokenize(input_text), return_pooled=True)
            cond = [[cond, {"pooled_output": pooled}]]
            negative_prompt = "extra body parts,bad quality, poor quality, disfigured, jpg, toy, bad anatomy, missing limbs, missing fingers, 3d, cgi"
            n_cond, n_pooled = clip.encode_from_tokens(clip.tokenize(negative_prompt), return_pooled=True)
            n_cond = [[n_cond, {"pooled_output": n_pooled}]]

            n_cond1 = node_helpers.conditioning_set_values(n_cond, {"start_percent": 0, "end_percent": 0.1})
            n_cond2 = zero_out(n_cond)
            n_cond2 = node_helpers.conditioning_set_values(n_cond2[0], {"start_percent": 0.1, "end_percent": 1.0})
            n_cond = n_cond1 + n_cond2

            seed = 1
            if seed == 0:
                seed = random.randint(0, 18446744073709551615)
            print(seed)
            sample = nodes.common_ksampler(model=model_patcher,
                                    seed=seed,
                                    steps=28,
                                    cfg=4.5,
                                    sampler_name="dpmpp_2m",
                                    scheduler="sgm_uniform",
                                    positive=cond,
                                    negative=n_cond,
                                    latent=latent,
                                    denoise=1)
            sample = sample[0]["samples"].to(torch.float16)
            vae.first_stage_model.cuda()
            decoded = vae.decode_tiled(sample).detach()


        Image1 = Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0])
        Image1.resize((512, 512))
        Image1.save("image1.png")

        #img to sticker
        response = requests.post('https://api.remove.bg/v1.0/removebg',
            files={'image_file': open('/content/TotoroUI/image1.png', 'rb')},
            data={'size': 'auto'},
            headers={'X-Api-Key': ''}, #ENTER YOUR REMOVEBG API KEY
        )
        if response.status_code == requests.codes.ok:
            with open('cutie2.png', 'wb') as out:
                out.write(response.content)
        else:
            print("Error:", response.status_code, response.text)
        image_path = '/content/TotoroUI/cutie2.png'

        if want_cap == "yes":
            def round_corner(radius, fill):
                corner = Image.new('RGB', (radius, radius), (0, 0, 0, 0))
                draw = ImageDraw.Draw(corner)
                draw.pieslice((0, 0, radius * 2, radius * 2), 180, 270, fill=fill)
                return corner

            def round_rectangle(size, radius, fill):
                """Draw a rounded rectangle"""
                width, height = size
                rectangle = Image.new('RGBA', size, fill)
                corner = round_corner(radius, fill)
                rectangle.paste(corner, (0, 0))
                rectangle.paste(corner.rotate(90), (0, height - radius))  # Rotate the corner and paste it
                rectangle.paste(corner.rotate(180), (width - radius, height - radius))
                rectangle.paste(corner.rotate(270), (width - radius, 0))
                return rectangle



            def get_font_size(textarea, text, font_name, pixel_gap=5, padding=10):
                text_width = textarea[0] - 2 * padding
                text_height = textarea[1]
                for point_size in range(90, 5, -1):
                    font = ImageFont.truetype(font_name, point_size)
                    avg_char_width = sum(font.getbbox(char)[2] for char in ascii_letters) / len(ascii_letters)
                    max_char_height = max(font.getbbox(char)[3] - font.getbbox(char)[1] for char in ascii_letters)

                    max_char_count = int(text_width / avg_char_width)
                    wrapped_text = textwrap.fill(text=text, width=max_char_count)
                    num_lines = len(wrapped_text.splitlines())

                    if (max_char_height * num_lines) + (pixel_gap * (num_lines + 1)) <= text_height:
                        return wrapped_text, point_size

                return "", 5  # Fallback to minimum size if no suitable size is found

            # Your original code
            imagee = Image.open(image_path)
            color = (255, 255, 255, 255)
            caption_area = (312, 50)
            padding = 0  # Adjust this value to increase/decrease padding
            caption5 = round_rectangle((312, 50), 10, (255,255,255,255))

            caption_text = context.user_data.get('caption', 'No caption')
            text = caption_text
            font_name = "" #(path to directory/comicbd.ttf)  # Ensure the font file is in the same directory as your script
            # Get wrapped text and appropriate font size
            wrapped_text, font_size = get_font_size(caption_area, text, font_name, padding=padding)

            # Draw text onto the caption
            draw = ImageDraw.Draw(caption5)
            font = ImageFont.truetype(font_name, font_size)

            # Center the text vertically and horizontally with padding
            text_bbox = draw.textbbox((0, 0), wrapped_text, font=font)
            text_width = text_bbox[2] - text_bbox[0]
            text_height = text_bbox[3] - text_bbox[1]
            x = padding + (caption_area[0] - 2 * padding - text_width) // 2
            y = (caption_area[1] - text_height-20) // 2
            draw.text((x, y), wrapped_text, fill="black", font=font)

            # Paste the caption onto the image
            imagee.paste(caption5, (100, 450), caption5)
            imagee.save('cutie2.png')
            img_path = '/content/TotoroUI/cutie2.png'
            cutie = Image.open(img_path)
            cutie = cutie.resize((512, 512))
            with open(img_path, 'rb') as sticker_file:
                await context.bot.send_sticker(chat_id=update.effective_chat.id, sticker=sticker_file)

        else:
            image_path = '/content/TotoroUI/cutie2.png'
            cutie = Image.open(image_path)
            cutie = cutie.resize((512, 512))
            with open(image_path, 'rb') as sticker_file:
                await context.bot.send_sticker(chat_id=update.effective_chat.id, sticker=sticker_file)

        await context.bot.delete_message(
            chat_id=loading_message.chat_id,
            message_id=loading_message.message_id
        )

        await query.edit_message_text("Here\'s your sticker :)  Thank you for your patience." )
        await context.bot.send_message(chat_id=update.effective_user.id, text="Use /download to download the sticker as image file")
        print("sticker sent")
        print("cancel confirm returns ask download 7")
        return 7#ASK_DOWNLOAD

    else:
        print("user clicked cancel")
        await query.edit_message_text('Sticker creation process has been *cancelled*.', parse_mode='Markdown')
        context.user_data.clear()
        return ConversationHandler.END

async def ask_download(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    print("ask download started")
    keyboard5 = [
        [InlineKeyboardButton("Yes", callback_data='yes')],
        [InlineKeyboardButton("No", callback_data='no')],
    ]
    reply_markup5 = InlineKeyboardMarkup(keyboard5)
    await update.message.reply_text("*Do you want to download your sticker as an image file?*", reply_markup=reply_markup5, parse_mode='Markdown')
    print("ask download returns download")
    return 8#DOWNLOAD

async def download(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    print("download started")
    query = update.callback_query
    await query.answer()
    want_img = query.data
    if want_img == 'yes':
        print("yes download clicked")
        await context.bot.send_document(chat_id=update.effective_chat.id,
                                        document=open("/content/TotoroUI/cutie2.png", 'rb'),
                                        filename='sticker.png')

        await query.edit_message_text(text="Here\'s your image file.\n*Help us improve - leave a rating for our bot! /rate*", parse_mode='Markdown')
        context.user_data.clear()
        return ConversationHandler.END
    else:
        print("no download clicked")
        await query.edit_message_text(text="*Help us improve - leave a rating for our bot! /rate*", parse_mode='Markdown')
        context.user_data.clear()
        return ConversationHandler.END




async def cancel(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    print("cancel started")
    await query.edit_message_text('Sticker creation process has been *cancelled*.', parse_mode='Markdown')
    context.user_data.clear()
    return ConversationHandler.END



async def feedback(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    print("feedback started")
    await update.message.reply_text('*Please provide your feedback:*', parse_mode='Markdown')
    print("feedback returns receive feedback 3")
    return 3

# Asynchronous message handler for feedback
async def receive_feedback(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    print("receive feedback started")
    user_id = update.effective_user.id
    feedback = update.message.text
    # Insert or update feedback in the database
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Insert or update feedback in the database
    cursor.execute('''
        INSERT OR REPLACE INTO feedback (user_id, feedback, timestamp)
        VALUES (?, ?, ?)
    ''', (user_id, feedback, timestamp))
    conn.commit()
    cursor.execute('SELECT * FROM feedback ORDER BY timestamp DESC LIMIT 1')
    feedback = cursor.fetchone()
    if feedback:
        feedback_text = f"Most Recent Feedback:\nUser ID: {feedback[0]}, Feedback: {feedback[1]}, Timestamp: {feedback[2]}"
        print(feedback_text)

    await update.message.reply_text('Thank you for your feedback! ')
    return ConversationHandler.END
keyboard4=[
    ["⭐⭐⭐⭐⭐","⭐⭐⭐⭐"],
    ["⭐⭐⭐","⭐⭐"],
    ["⭐"]
    ]

reply_markup4= ReplyKeyboardMarkup(keyboard4, resize_keyboard=True, one_time_keyboard=True)
# Asynchronous command handler for /rate
async def rate(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    print("rate started")
    await update.message.reply_text('*Please provide rating:*',reply_markup=reply_markup4, parse_mode='Markdown')
    print("rate returns button")  
    return 19

async def button(update: Update, context: ContextTypes.DEFAULT_TYPE):
    print("button started")
    await context.bot.send_message(chat_id=update.effective_chat.id, text="Thankyou for your rating. You can give your valuable feedback using /feedback or you can /skip this")
    print("button returns feedback 30 or skip")
    return 30



async def skip(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    print("skip started")
    await context.bot.send_message(chat_id=update.effective_chat.id, text='Thank you! You can continue creating exciting stickers!')
    return ConversationHandler.END







#some confused users might try to send commands to the bot that it doesn't understand, so you can use a MessageHandler with a COMMAND filter to reply to all commands that were not recognized by the previous handlers
#IMPORTANT: This handler must be added last.
async def unknownCommand(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await context.bot.send_message(chat_id=update.effective_chat.id, text="Sorry, I didn't understand that command:(")
async def unknownText(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await context.bot.send_message(chat_id=update.effective_chat.id, text="Sorry, I didn't understand that what you want to say:(.")







if __name__ == '__main__':
    application = ApplicationBuilder().token('').build() #ENTER TELEGRAM BOT TOKEN NO.

    conv_handler3 = ConversationHandler(
        entry_points=[CommandHandler('start',start)],
        states={
            1: [CallbackQueryHandler(trending)],
            7: [CommandHandler('download', ask_download)],
            8: [CallbackQueryHandler(download)],
        },
        fallbacks=[CommandHandler('cancel', cancel)],
        allow_reentry=True

    )

    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('addtext', addtext)],
        states={
            0: [MessageHandler(filters.TEXT & ~filters.COMMAND, receive_text)],
            1: [CommandHandler('style', style)],
            2: [CallbackQueryHandler(receive_style)],
            3: [CallbackQueryHandler(ask_caption)],
            4:[MessageHandler(filters.TEXT & ~filters.COMMAND, get_caption)],
            5:[CommandHandler('preview', preview)],
            6: [CallbackQueryHandler(cancel_confirm)],
            7: [CommandHandler('download', ask_download)],
            8:  [CallbackQueryHandler(download)],

        },
        fallbacks=[CommandHandler('cancel', cancel)],
        allow_reentry=True

    )

    conv_handler2 = ConversationHandler(
        entry_points=[CommandHandler('rate',rate)],
        states={
            19: [MessageHandler(filters.TEXT, button)],
            30: [CommandHandler('feedback', feedback)],
            3: [MessageHandler(filters.TEXT & ~filters.COMMAND, receive_feedback)]

        },
        fallbacks=[],
        allow_reentry=True

    )




    start_handler = CommandHandler('start', start)
    trending_handler = CommandHandler('trending', trending)
    helps_handler = CommandHandler('help', helps)
    addtext_handler = CommandHandler('addtext', addtext)
    askdownload_handler = CommandHandler('download', ask_download)
    style_handler = CommandHandler('style', style)
    preview_handler = CommandHandler('preview', preview)
    cancel_handler = CommandHandler('cancel', cancel)
    skip_handler = CommandHandler('skip', skip)



    application.add_handler(conv_handler)
    application.add_handler(conv_handler3)
    application.add_handler(start_handler)
    application.add_handler(helps_handler)
    application.add_handler(trending_handler)
    application.add_handler(conv_handler2)
    application.add_handler(skip_handler)
    application.add_handler(addtext_handler)
    application.add_handler(style_handler)
    application.add_handler(preview_handler)
    application.add_handler(askdownload_handler)
    application.add_handler(cancel_handler)


    unknown_command_handler = MessageHandler(filters.COMMAND, unknownCommand)
    application.add_handler(unknown_command_handler)
    unknown_text_handler = MessageHandler(filters.TEXT, unknownText)
    application.add_handler(unknown_text_handler)

    application.run_polling()



/tmp/ipykernel_34/235974272.py:647: PTBUserWarning: If 'per_message=False', 'CallbackQueryHandler' will not be tracked for every message. Read this FAQ entry to learn more about the per_* settings: https://github.com/python-telegram-bot/python-telegram-bot/wiki/Frequently-Asked-Questions#what-do-the-per_-settings-in-conversationhandler-do.
  conv_handler3 = ConversationHandler(
/tmp/ipykernel_34/235974272.py:659: PTBUserWarning: If 'per_message=False', 'CallbackQueryHandler' will not be tracked for every message. Read this FAQ entry to learn more about the per_* settings: https://github.com/python-telegram-bot/python-telegram-bot/wiki/Frequently-Asked-Questions#what-do-the-per_-settings-in-conversationhandler-do.
  conv_handler = ConversationHandler(


started start 
start return trending 9 or add text
started add text
add text return receive text
receive text started
receive text return style 1
style started
style return receive style
receive style started
receive style returns ask caption 3
ask caption started
ask caption returns get caption
get caption started
get caption returns preview
preview started
preview returns cancel confirm
cancel confirm started
user clicked confirm
1


  0%|          | 0/28 [00:00<?, ?it/s]

sticker sent
cancel confirm returns ask download 7
ask download started
ask download returns download
download started
yes download clicked
rate started
rate returns button
button started
button returns feedback 30 or skip
feedback started
feedback returns receive feedback 3
receive feedback started
Most Recent Feedback:
User ID: 6291680713, Feedback: good, Timestamp: 2024-07-27 07:39:50


RuntimeError: Cannot close a running event loop

In [8]:
cursor.execute('SELECT * FROM feedback')
feedbacks = cursor.fetchall()

print("User Feedbacks:")
for feedback in feedbacks:
    print(f"User ID: {feedback[0]}, Feedback: {feedback[1]} , TIME: {feedback[2]}" )

User Feedbacks:
User ID: 6291680713, Feedback: good , TIME: 2024-07-27 07:39:50
